In [47]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [48]:
num_nodes = 10
dim = 100

In [49]:
embedding1 = nn.Embedding(num_nodes, dim)
embedding2 = nn.Embedding(num_nodes, dim)
linear1 = nn.Linear(dim, dim)

linear2 = nn.Linear(dim, dim)

In [50]:
_k = 10
_alpha = 3

In [54]:
nodevec1 = embedding1(torch.LongTensor([0]))
nodevec2 = embedding1(torch.LongTensor([5,6,7,8,9]))

In [57]:
embedding1.weight[0]

tensor([ 0.6674,  0.6622,  0.9878, -0.9403, -1.7023, -0.0283,  0.9315,  1.0906,
        -1.3533,  0.2778,  0.2493, -0.4917, -1.0677,  1.1623, -1.0099,  0.0951,
         0.8180, -0.9431, -0.7786,  0.9390, -0.7209, -0.7910, -1.0337,  0.4590,
         2.5688, -1.1732, -0.2422, -0.2101, -1.5261,  0.1269, -0.4141,  1.0866,
         0.9579, -2.3207, -0.6144,  0.8565, -2.4947,  0.0157,  0.2255, -0.6350,
         0.0915,  1.5943, -0.7641, -1.5641,  0.1758, -0.0717, -1.2744, -0.2095,
        -0.4861, -1.1992, -0.1543, -1.1047, -2.2990,  0.6728,  1.3440, -0.6964,
         1.2998,  0.5202,  1.0204,  0.9395, -1.7302, -0.0508,  0.8373, -1.0504,
        -0.3942,  0.5006,  0.3554,  0.2874,  0.3330, -0.3154, -1.3336,  0.9922,
        -0.9262,  0.0301, -0.8390,  0.7555,  0.4789, -0.5742,  0.2506, -0.9724,
         0.6120,  0.4797,  2.3451,  0.0123,  0.5873, -0.5873, -0.6855,  0.6388,
         0.2004, -0.5277,  0.4636,  0.4679,  0.7206, -1.3038, -0.6950,  0.1445,
         0.0926, -0.6763,  0.2900,  0.30

In [55]:
nodevec1

tensor([[ 0.6674,  0.6622,  0.9878, -0.9403, -1.7023, -0.0283,  0.9315,  1.0906,
         -1.3533,  0.2778,  0.2493, -0.4917, -1.0677,  1.1623, -1.0099,  0.0951,
          0.8180, -0.9431, -0.7786,  0.9390, -0.7209, -0.7910, -1.0337,  0.4590,
          2.5688, -1.1732, -0.2422, -0.2101, -1.5261,  0.1269, -0.4141,  1.0866,
          0.9579, -2.3207, -0.6144,  0.8565, -2.4947,  0.0157,  0.2255, -0.6350,
          0.0915,  1.5943, -0.7641, -1.5641,  0.1758, -0.0717, -1.2744, -0.2095,
         -0.4861, -1.1992, -0.1543, -1.1047, -2.2990,  0.6728,  1.3440, -0.6964,
          1.2998,  0.5202,  1.0204,  0.9395, -1.7302, -0.0508,  0.8373, -1.0504,
         -0.3942,  0.5006,  0.3554,  0.2874,  0.3330, -0.3154, -1.3336,  0.9922,
         -0.9262,  0.0301, -0.8390,  0.7555,  0.4789, -0.5742,  0.2506, -0.9724,
          0.6120,  0.4797,  2.3451,  0.0123,  0.5873, -0.5873, -0.6855,  0.6388,
          0.2004, -0.5277,  0.4636,  0.4679,  0.7206, -1.3038, -0.6950,  0.1445,
          0.0926, -0.6763,  

In [43]:
nodevec1 = torch.tanh(_alpha * linear1(nodevec1))
nodevec2 = torch.tanh(_alpha * linear2(nodevec2))

In [44]:
nodevec1.shape

torch.Size([5, 100])

In [45]:
a = torch.mm(nodevec1, nodevec2.transpose(1, 0)) - torch.mm(
            nodevec2, nodevec1.transpose(1, 0)
        )

In [46]:
a.shape

torch.Size([5, 5])

In [38]:
a

tensor([[0.]], grad_fn=<SubBackward0>)